<a href="https://colab.research.google.com/github/vedant75/Gpt/blob/main/Gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-11-12 10:48:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-11-12 10:48:56 (23.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.__version__

'2.8.0+cu126'

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# Extract the unique chars from the text file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## We are doing simple converting of text to nums. There are alternate available:
* **Sentencepiece**: Use by Google
* **Tiktoken**: Use by OpenAi

In [6]:
# converting the words for passing in nn
stoi = {v:i for i, v in enumerate(chars)}
itos = {v:i for i, v in stoi.items()}
encode = lambda s: [stoi[i] for i in s]
decode = lambda s: ''.join([itos[i] for i in s])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


# Store the data into torch.tensor

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(len(data))

torch.Size([1115394]) torch.int64
1115394


# Split the data into train & Val
 * Split will be 90 - 10 into train & val

In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
# Since we can't pass entire data as it in transformer as it will be computationally expensive, we prefer to pass it in chunks
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
X = train_data[:block_size]
y = train_data[1: block_size+1]
X, y

(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [11]:
for i in range(block_size):
  context = X[:i+1]
  next_ele = y[i]
  print(context, next_ele)

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [29]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(len(data)- block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print(xb.shape, yb.shape)
print(xb)
print(yb)

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the t

In [40]:
vocab_size
torch.manual_seed(42)
emb = nn.Embedding(vocab_size, vocab_size)
dumy = torch.tensor([[1,2], [1,2]])
print(emb(dumy).shape)
x = emb(dumy)
x[:, -1, :]

torch.Size([2, 2, 65])


tensor([[-1.4364, -1.1299, -0.1360,  1.6354,  0.6547,  0.5760,  1.1415,  0.0186,
         -1.8058,  0.9254, -0.3753,  1.0331, -0.6867,  0.6368, -0.9727,  0.9585,
          1.6192,  1.4506,  0.2695, -0.2104, -0.7328,  0.1043,  0.3488,  0.9676,
         -0.4657,  1.6048, -2.4801, -0.4175, -1.1955,  0.8123, -1.9006,  0.2286,
          0.0249, -0.3460,  0.2868, -0.7308,  0.1748, -1.0939, -1.6022,  1.3529,
          1.2888,  0.0523, -1.5469,  0.7567,  0.7755,  2.0265,  0.0358,  0.1206,
         -0.8057, -0.2076, -0.9319, -1.5910, -1.1360, -0.5226, -0.5188, -1.5013,
         -1.9267,  0.1279,  1.0229, -0.5558,  0.7043,  0.7099,  1.7744, -0.9216,
          0.9624],
        [-1.4364, -1.1299, -0.1360,  1.6354,  0.6547,  0.5760,  1.1415,  0.0186,
         -1.8058,  0.9254, -0.3753,  1.0331, -0.6867,  0.6368, -0.9727,  0.9585,
          1.6192,  1.4506,  0.2695, -0.2104, -0.7328,  0.1043,  0.3488,  0.9676,
         -0.4657,  1.6048, -2.4801, -0.4175, -1.1955,  0.8123, -1.9006,  0.2286,
         

In [42]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        # Since pytorch cross_entropy expects data into (multidim, C) -> in case of multi dims.
        # We need to change the shape of our logits
        if targets is None:
          loss = None
        else:
          B, T, C = logits.shape
          logits = logits.view(B*T, C)
          targets = targets.view(B*T) # eq: targets.view(-1)

          # Calculate loss
          loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
      for _ in range(max_new_tokens):
        # idx will be the shape of (B, T)
        logits, loss = self(idx)
        # focus only on the last time stamp
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # becomes (B, C)
        # apply the multinomial distribution
        idx_next = torch.multinomial(probs, num_samples=1)
        # append sample indexed to the current index
        idx = torch.cat((idx, idx_next), dim=1)
      return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [43]:
# Add the optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [45]:
batch_size = 32
for epoch in range(10000):
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if epoch%1000 == 0:
    print(f'epoch: {epoch} || Loss: {loss.item()}')


epoch: 0 || Loss: 2.594559907913208
epoch: 1000 || Loss: 2.5389461517333984
epoch: 2000 || Loss: 2.5308570861816406
epoch: 3000 || Loss: 2.649641752243042
epoch: 4000 || Loss: 2.466621160507202
epoch: 5000 || Loss: 2.487251043319702
epoch: 6000 || Loss: 2.5632574558258057
epoch: 7000 || Loss: 3.117689609527588
epoch: 8000 || Loss: 2.380329132080078
epoch: 9000 || Loss: 2.3261146545410156


In [47]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


h wsthe.
Ove, g
O:
YCI'st morel pouromaveak y n'd dge lfrevetoutourd t,
An bevinds tr
Yound mofay athe swh counswowhes!

V: theniecy's berd t wiles saprdge He h myor s hy; hoour.

Anseg ou izeprr pr ingivequce sme men Ppen

wst;
Sed? Cid ar, at nous
ISSSingeli'sllos, ts fllainonerey t wanonthe m me f ndle spind are ilmathe he hbed agom imisesh NAnd Ge HELey yFone sthithefridwo Yourd o, thetay umapllfio 'l sit ngrfr mpot wnven: Cllexfomatot; bbu ninouspant'TH:
OMy wind;
3im is'lofr y fol y d augo


In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

# Full finished code

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4196, val loss 4.4025
step 100: train loss 2.8220, val loss 2.8232
step 200: train loss 2.6195, val loss 2.6374
step 300: train loss 2.5521, val loss 2.5671
step 400: train loss 2.5259, val loss 2.5463
step 500: train loss 2.4643, val loss 2.4592
step 600: train loss 2.4367, val loss 2.4396
step 700: train loss 2.4006, val loss 2.4037
step 800: train loss 2.3828, val loss 2.3767
step 900: train loss 2.3532, val loss 2.3731
step 1000: train loss 2.3564, val loss 2.3702
step 1100: train loss 2.3066, val loss 2.3059
step 1200: train loss 2.2859, val loss 2.3110
step 1300: train loss 2.2779, val loss 2.2819
step 1400: train loss 2.2359, val loss 2.2625
step 1500: train loss 2.2457, val loss 2.2534
step 1600: train loss 2.2235, val loss 2.2432
step 1700: train loss 2.1851, val loss 2.2333
step 1800: train loss 2.1842, val loss 2.2044
step 1900: train loss 2.1604, val loss 2.1827
step 2000: train loss 2.1652, val loss 2.1759
step 2100: train loss 2.